In [12]:
import torch
import torch.nn as nn
# import torchvision.datasets as normal_datasets
# import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.autograd import Variable
from pathlib import Path
import pandas as pd
import numpy as np
import os

# Pre-processing

In [37]:
class SHM_Dataset(Dataset):
    """ Prepare dataset for pytorch

    Parameters
    ----------
    Dataset : _type_
        _description_
    """

    def __init__(self, data_file):
        self.data_file = Path(data_file)
        # self.data_df = pd.read_csv(self.data_file)
        self.data_df = pd.read_json(self.data_file)

shmDS = SHM_Dataset("~/Codes/homework/SHM/shm01s.json")

In [38]:
shmDS.data_df

,C_1_CH_1_frames,C_1_CH_1_feature_matrixs,C_1_CH_1_labels,C_1_CH_2_frames,C_1_CH_2_feature_matrixs,C_1_CH_2_labels,C_1_CH_3_frames,C_1_CH_3_feature_matrixs,C_1_CH_3_labels,C_1_CH_4_frames,...,C_1_CH_13_labels,C_1_CH_14_frames,C_1_CH_14_feature_matrixs,C_1_CH_14_labels,C_1_CH_15_frames,C_1_CH_15_feature_matrixs,C_1_CH_15_labels,C_1_CH_16_frames,C_1_CH_16_feature_matrixs,C_1_CH_16_labels
0,"[[0.0026424], [0.0027374], [0.0024156], [0.002...","[[0.0026424, 0.0027374, 0.0024156, 0.002301, 0...",C_1_CH_1,"[[-0.0020818], [-0.0021144], [-0.0021016], [-0...","[[-0.0020818, -0.0021144, -0.0021016, -0.00242...",C_1_CH_2,"[[0.0005269], [0.00047746], [0.0008568], [0.00...","[[0.0005269, 0.00047746, 0.0008568, 0.00071764...",C_1_CH_3,"[[-0.0013643], [-0.00125292], [-0.00046494], [...",...,C_1_CH_13,"[[0.00179798], [0.0020139999999999997], [0.002...","[[0.00179798, 0.0020139999999999997, 0.0026915...",C_1_CH_14,"[[-0.00051988], [-0.00011429], [-0.00051378], ...","[[-0.00051988, -0.00011429, -0.00051378, -0.00...",C_1_CH_15,"[[-0.7602166], [-0.16169114], [-0.13078156], [...","[[-0.7602166, -0.16169114, -0.13078156, 0.0079...",C_1_CH_16
1,"[[0.0023826], [0.0023325999999999998], [0.0022...","[[0.0023826, 0.0023325999999999998, 0.0022704,...",C_1_CH_1,"[[-0.0020256], [-0.002376], [-0.00209819999999...","[[-0.0020256, -0.002376, -0.002098199999999999...",C_1_CH_2,"[[0.00041642000000000003], [0.00031602], [0.00...","[[0.00041642000000000003, 0.00031602, 0.000802...",C_1_CH_3,"[[-0.00052446], [-0.00069536], [-0.00145984], ...",...,C_1_CH_13,"[[-5.661e-05], [-0.0005216999999999999], [-0.0...","[[-5.661e-05, -0.0005216999999999999, -0.00139...",C_1_CH_14,"[[6.363e-05], [-0.0022083999999999997], [-0.00...","[[6.363e-05, -0.0022083999999999997, -0.003801...",C_1_CH_15,"[[0.47074742], [-0.43908696], [-0.24686662], [...","[[0.47074742, -0.43908696, -0.24686662, 0.2694...",C_1_CH_16
2,"[[0.0023778], [0.0023464], [0.002307], [0.0023...","[[0.0023778, 0.0023464, 0.002307, 0.002307, 0....",C_1_CH_1,"[[-0.0022152], [-0.00218], [-0.0023732], [-0.0...","[[-0.0022152, -0.00218, -0.0023732, -0.0022512...",C_1_CH_2,"[[0.00053238], [0.00033066], [0.00084306], [0....","[[0.00053238, 0.00033066, 0.00084306, 0.001132...",C_1_CH_3,"[[-0.0005748], [-0.00100236], [-0.0019768], [-...",...,C_1_CH_13,"[[-0.0027183999999999997], [-0.003117], [-0.00...","[[-0.0027183999999999997, -0.003117, -0.002084...",C_1_CH_14,"[[-0.00152118], [-8.713e-05], [0.0020846], [0....","[[-0.00152118, -8.713e-05, 0.0020846, 0.001514...",C_1_CH_15,"[[-0.015452814], [0.4918511], [-0.7081984], [0...","[[-0.015452814, 0.4918511, -0.7081984, 0.20465...",C_1_CH_16
3,"[[0.0023936], [0.0023997999999999997], [0.0023...","[[0.0023936, 0.0023997999999999997, 0.00233479...",C_1_CH_1,"[[-0.0021154], [-0.0025269999999999997], [-0.0...","[[-0.0021154, -0.0025269999999999997, -0.00211...",C_1_CH_2,"[[0.00086594], [0.00046708000000000003], [0.00...","[[0.00086594, 0.00046708000000000003, 0.000731...",C_1_CH_3,"[[-0.0013316600000000001], [-0.00179644], [-0....",...,C_1_CH_13,"[[-0.002502], [-0.00095598], [0.00182666000000...","[[-0.002502, -0.00095598, 0.001826660000000000...",C_1_CH_14,"[[0.00161272], [0.0020458], [0.00109484], [0.0...","[[0.00161272, 0.0020458, 0.00109484, 0.0006062...",C_1_CH_15,"[[-0.36295168], [0.7594261999999999], [1.00667...","[[-0.36295168, 0.7594261999999999, 1.0066732, ...",C_1_CH_16
4,"[[0.0023631999999999998], [0.0023406], [0.0023...","[[0.0023631999999999998, 0.0023406, 0.002346, ...",C_1_CH_1,"[[-0.0024296], [-0.0020266], [-0.0024162], [-0...","[[-0.0024296, -0.0020266, -0.0024162, -0.00225...",C_1_CH_2,"[[0.00050858], [0.00043138000000000003], [0.00...","[[0.00050858, 0.00043138000000000003, 0.000482...",C_1_CH_3,"[[-0.00109178], [-0.00139148], [-0.00139972000...",...,C_1_CH_13,"[[-0.00184008], [0.0007017600000000001], [0.00...","[[-0.00184008, 0.0007017600000000001, 0.002521...",C_1_CH_14,"[[0.0013191400000000001], [0.00019363600000000...","[[0.0013191400000000001, 0.00019363600000

In [40]:
a = shmDS.data_df.iloc[0, 1]
aa = np.array(a)
aa.shape

(8, 16)

In [17]:
class FaceLandmarksDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:].as_matrix()
        landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample


face_dataset = FaceLandmarksDataset(csv_file='faces/face_landmarks.csv',
                                    root_dir='faces/')

for i in range(len(face_dataset)):
    sample = face_dataset[i]
    print(sample['image'])
    print(i,sample['image'].shape, sample['landmarks'].shape)


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image),
                'landmarks': torch.from_numpy(landmarks)}

## CNN Model

In [18]:
# Two-layer convolution
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Quick build with sequence tools
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7 * 7 * 32, 10)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = out.view(out.size(0), -1)  # reshape
        out = self.fc(out)
        return out

cnn = CNN()

## Loss and Optimizer

In [19]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

## Train the model

In [20]:
# Hyper Parameters
num_epochs = 1
batch_size = 100
learning_rate = 0.001

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)

        # Forward + Backward + Optimize 
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f'
                  % (epoch + 1, num_epochs, i + 1, len(train_dataset) // batch_size, loss.item()))

Epoch [1/1], Iter [100/600] Loss: 0.1478
Epoch [1/1], Iter [200/600] Loss: 0.1267
Epoch [1/1], Iter [300/600] Loss: 0.0246
Epoch [1/1], Iter [400/600] Loss: 0.0775
Epoch [1/1], Iter [500/600] Loss: 0.0264
Epoch [1/1], Iter [600/600] Loss: 0.0119


## Test models

In [21]:
cnn.eval()  # Change to test form, application scenarios such as: dropout
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images)
    labels = Variable(labels)

    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.data).sum()

print(' Test Accuracy: %d %%' % (100 * correct / total))

# Save the Trained Model
torch.save(cnn.state_dict(), 'cnn.pkl')

 Test Accuracy: 98 %
